In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Program, Task
import pandas as pd
from dataclasses import replace

In [2]:
# CHANGE ME!
TASK_ID = 'average_window'
AUTHOR = 'scott'

In [3]:
task = Task(
    id=TASK_ID,
    name="Windowed average",
    description="compute average of data points in each disjoint time window of width 7 ordered by time",
    category='Time Series',
    plan=[{
        "id": "key",
        "description": "of width 7",
    }, {
        "id": "group by",
        "description": "points in each disjoint time window",
    }, {
        "id": "average",
        "description": "compute average",
    }, {
        "id": "output_order",
        "description": "ordered by time",
    }],
    sample_input={
        "data": [
            {"time": 20, "x": 14.0},
            {"time": 19, "x": 15.0},
            {"time": 2, "x": 3.0},
            {"time": 1, "x": 2.0},
            {"time": 3, "x": 7.0},
            {"time": 10, "x": 9.0},
            {"time": 11, "x": 11.0},
        ]
    },
    sample_output=[4.0, 10.0, 14.5]
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [15]:
q = replace(prototype,
    language='q',
    source='''average_window: {[data]
  (value select avg x by 7 xbar time from data) `x}''').load_plan()
q.execute(task, debug=True)
q.save()

/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmp56461h_3
data:("IF"; enlist ",") 0:`:/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmp56461h_3/data.csv
average_window: {[data]
  (value select avg x by 7 xbar time from data) `x}

output: average_window[data]
output: ([] c:output)
output: (asc cols output) xcols output
save `:output.csv



In [16]:
q.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "q", "plan": {}, "source": "average_window: {[data]…

In [7]:
r = replace(prototype,
    language='r',
    source='''average_window <- function(data) {
  data %>% 
    group_by(floor(time / 7)) %>% 
    summarize(avg = mean(x)) %>% 
    pull(avg)
}''').load_plan()
r.execute(task)
r.save()

`summarise()` ungrouping output (override with `.groups` argument)


In [8]:
r.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "r", "plan": {}, "source": "average_window <- funct…

In [77]:
python_imp = replace(prototype,
    language='python-imperative',
    implementation='',
    source='''def average_window(data):
  def window(value):
    return math.floor(value["time"] / 7)
  if len(data) < 1:
    return []
  data.sort(key=lambda v: v["time"])
  result = []
  current_window = window(data[0])
  total, count = data[0]["x"], 1
  for value in data[1:]:
    time_window = window(value)
    if time_window != current_window:
      result.append(total / count)
      current_window, total, count = time_window, 0, 0
    total += value["x"]
    count += 1
  result.append(total / count)
  return result''').load_plan()
python_imp.execute(task)
python_imp.save()

In [72]:
python_fun = replace(prototype,
    language='python-functional',
    implementation='',
    source='''def average_window(data):
  def window(value):
    return math.floor(value["time"] / 7)
  grouped_values = [
    [point["x"] for point in data if window(point) == w]
    for w in set(map(window, data))
  ]
  return [
    sum(values) / len(values)
    for values in grouped_values
  ]''').load_plan()
python_fun.execute(task)
python_fun.save()

In [75]:
pandas = replace(prototype,
    language='python-pandas',
    implementation='',
    source='''def average_window(data):
  def window(t):
    return math.floor(t / 7)
  result = (data.sort_values("time")
                .set_index("time").groupby(window).mean())
  return result['x'].tolist()''').load_plan()
pandas.execute(task)
pandas.save()

In [52]:
sql = replace(prototype,
    language='sql',
    source='''SELECT AVG(x) as x
FROM data
GROUP BY cast(time / 7 as int)
ORDER BY time''').load_plan()
sql.execute(task)
sql.save()

In [82]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl window(w: number)
window(t/7) :- data(t, _).
.decl windowed(w: number, x: float)
windowed(t/7, x) :- data(t, x).
.decl windowed_total(w: number, x: float)
windowed_total(w, total / n) :- window(w),
  total = sum x : { windowed(w, x) },
      n = sum z : { windowed(w, x), z=1.0 }.
average_window(v) :- windowed_total(_, v).''').load_plan()
datalog.execute(task, debug=False)
datalog.save()

In [66]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "sql", "plan": {"key": [{"line": 2, "start": 9, "en…

In [83]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "datalog", "plan": {"average": [{"line": 5, "start"…

In [87]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "python-pandas", "plan": {"key": [{"line": 1, "star…

In [88]:
python_fun.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "python-functional", "plan": {"output_order": [{"li…

In [89]:
python_imp.widget(task)

Output()

CodeWidget(program='{"task": "average_window", "language": "python-imperative", "plan": {"key": [{"line": 1, "…